# Задание 7

## 1. PropertyCreator (0.2 балла)

Напишите мета класс для создания свойств (property) класса из функций начинающихся с "set\_", "get\_" или "del_". Пример использования:
<code>
    
class TestPropertyCreator(metaclass=PropertyCreator):
    
    def \__init\__(self, lo):
        self.__x = None
        self.lo = lo

    def get_x(self):
        return self.__x

    def set_x(self, value):
        if value < self.lo:
            raise ValueError("Value must in condition: {} <= value".format(self.lo))
        self.__x = value
    
    def del_x(self):
        self.__x = "No more"

    pass


obj = TestPropertyCreator(5)
obj.x = 4
print(obj.x)
del (obj.x)
</code>

* Мета класс должен поддерживать наследование, в смысле создавать свойства у потомков.
* Должен поддерживать частичное описание свойств, т. е. например, описание одного метода get_val (без set_val и del_val).
* Поддерживать множественное использование одного свойства с одним именем в разных классах:

<code>
class A(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class A"

class B(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class B"

class C(metaclass=PropertyCreator):
    def set_x(self, value):
        self.value = "x in class C"
    def get_x(self):
        return self.value
</code>
* Должен уметь обрабатывать имен с несколькими подчеркиваниями "get_raw_text".

#### Решение

In [1]:
from collections import defaultdict

In [2]:
class PropertyCreator(type):
    def __new__(cls, name, bases, attrs):
        props = defaultdict(lambda: {'get': None, 'set': None, 'del': None})
        new_attrs = {}
        
        for attr, value in attrs.items():
            if attr.startswith('get_'):
                props[attr[4:]]['get'] = value
            elif attr.startswith('set_'):
                props[attr[4:]]['set'] = value
            elif attr.startswith('del_'):
                props[attr[4:]]['del'] = value
            else:
                new_attrs[attr] = value
        
        for prop, value in props.items():
            new_attrs[prop] = property(value['get'], value['set'], value['del'])
        
        return super(PropertyCreator, cls).__new__(cls, name, bases, new_attrs)

#### Протестируйте свое решение

In [3]:
def test_simple():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self, lo):
            self.__x = None
            self.lo = lo
            
        def get_x(self):
            return self.__x

        def set_x(self, value):
            if value < self.lo:
                raise ValueError("Value must in condition: {} <= value".format(self.lo))
            self.__x = value

        def del_x(self):
            self.__x = "No more"
    
    obj = TestPropertyCreator(3)
    obj.x = 4
    print(obj.x)
    
    try:
        obj.x = 2
    except ValueError as e:
        print(e)
    
    del(obj.x)
    print(obj.x)

    
test_simple()

4
Value must in condition: 3 <= value
No more


In [4]:
def test_with_inheritance():
    class TestPropertyCreator(metaclass=PropertyCreator):
        pass

    class TestPropertyCreatorInheritance(TestPropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_x(self, value):
            self._secret_list.append("set")
    
    obj = TestPropertyCreatorInheritance()
    print(obj.x)
    obj.x = 5
    
    try:
        obj.x = 2
    except ValueError as e:
        print(e)
    
    print(obj.x)
    print(obj._secret_list)
    

test_with_inheritance()

0
0
['get', 'set', 'set', 'get']


In [5]:
def test_partially_defined():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_y(self, value):
            self._secret_list.append("set")
            self._y = value
        
    obj = TestPropertyCreator()
    obj.y = 2
    print(obj.x)
    
    try:
        obj.y = 3
    except ValueError as e:
        print(e)
        
    print(obj.x)
    print(obj._secret_list)
    
    try:
        obj.x = 3
    except Exception as e:
        print(e)
    
    
test_partially_defined()

0
0
['set', 'get', 'set', 'get']
can't set attribute


In [6]:
def test_sanity():
    class TestPropertyCreator(metaclass=PropertyCreator):
        _text = 0
        def get_raw_text(self):
             return self._boo

        def get_text(self):
             return self._text % 2

        def set_text(self, value):
            try:
                self._text = int(value)
            except ValueError:
                raise TypeError("unproper value for text: {}".format(value))
             
    obj = TestPropertyCreator()
    
    try:
        obj.raw_text = 'raw'
    except Exception as e:
        print(e)
        
    try:
        print(obj.raw_text)
    except Exception as e:
        print(e)
        
    print(obj.text)
    
    obj.text = 11
    print(obj.text)
    

test_sanity()

can't set attribute
'TestPropertyCreator' object has no attribute '_boo'
0
1


In [7]:
def test_multiple_usages():
    class TestPropertyCreatorA(metaclass=PropertyCreator):
        def get_x(self):
            return 0
    class TestPropertyCreatorB(metaclass=PropertyCreator):
        def get_x(self):
            return 1
    class TestPropertyCreatorC(metaclass=PropertyCreator):
        def set_x(self, value):
            self.value = value + 1
        def get_x(self):
            return self.value
    
    obj_A = TestPropertyCreatorA()
    obj_B = TestPropertyCreatorB()
    obj_C = TestPropertyCreatorC()
    
    print(obj_A.x)
    print(obj_B.x)
    
    obj_C.x = 5
    print(obj_C.x)
    
test_multiple_usages()

0
1
6


## 2. InstanceCountExceptioner (0.2 балла)
Напишите метакласс InstanceCountExeptioner, который будет следить за количеством экземпляров класса, использующих его. Количество задается через поле класса \_\_max_instance\_count\_\_. Т. е. число экземпляров каждого класса регулируется отдельно. Если в классе не указано поле \_\_max_instance\_count\_\_, то используйте заранее заданное число в метаклассе (любое). Пример:

<code>
class TestInstanceCountExceptionerA(metaclass=InstanceCountExeptioner):
    \_\_max_instance\_count\_\_ = 2
    def \__init\__(self, a):
        self.a = a


class TestInstanceCountExceptionerB(metaclass=InstanceCountExeptioner):
    \_\_max_instance\_count\_\_ = 1
    def \__init\__(self, a):
        self.a = a

a_one = TestInstanceCountExceptionerA('one')
a_two = TestInstanceCountExceptionerA('two')
b_one = TestInstanceCountExceptionerB('one')
\# пока всё шло хорошо

\# а вот
a_three = TestInstanceCountExceptionerA('three')
\# выкенет исключение InstanceCountException (ваше собственное исключение)
</code>

#### Решение

In [8]:
from collections import Counter

In [9]:
class InstanceCountException(Exception):
    def __init__(self, *args, **kwargs):
        Exception.__init__(self, *args, **kwargs)

In [10]:
class InstanceCountExceptioner(type):
    max_instance_count = defaultdict(int)
    cnt = Counter()
    
    def __new__(cls, name, bases, attrs):
        cls.cnt[name] = 0
        cls.max_instance_count[name] = attrs.get('__max_instance_count__', 10)
        return super(InstanceCountExceptioner, cls).__new__(cls, name, bases, attrs)
        
    def __call__(cls, *args, **kwargs):
        name = cls.__name__
        
        if cls.max_instance_count[name] <= cls.cnt[name]:
            raise InstanceCountException('Reached max number of instances')
            
        cls.cnt[name] += 1
        return super(InstanceCountExceptioner, cls).__call__(*args, **kwargs)

#### Протестируйте свое решение

In [11]:
def test_simple():
    
    class TestInstanceCountExceptionerA(metaclass=InstanceCountExceptioner):
        __max_instance_count__ = 2

        def __init__(self):
            self.a = 1

        def get(self):
            return self.a


    class TestInstanceCountExceptionerB(metaclass=InstanceCountExceptioner):
        __max_instance_count__ = 3

        def __init__(self):
            self.b = 2

        def get(self):
            return self.b
        
    a_one = TestInstanceCountExceptionerA()
    b_one = TestInstanceCountExceptionerB()
    
    
test_simple()

In [12]:
def test_create():
    class TestInstanceCountExceptionerA(metaclass=InstanceCountExceptioner):
        __max_instance_count__ = 2

        def __init__(self):
            self.a = 1

        def get(self):
            return self.a


    class TestInstanceCountExceptionerB(metaclass=InstanceCountExceptioner):
        __max_instance_count__ = 3

        def __init__(self):
            self.b = 2

        def get(self):
            return self.b
        
    a_one = TestInstanceCountExceptionerA()
    b_one = TestInstanceCountExceptionerB()
    
    print(a_one.get())
    print(b_one.get())
    print(a_one.__max_instance_count__)
    print(b_one.__max_instance_count__)
    
    
test_create()

1
2
2
3


In [13]:
def test_fail_create_a():
    class TestInstanceCountExceptionerA(metaclass=InstanceCountExceptioner):
        __max_instance_count__ = 2

        def __init__(self):
            self.a = 1

        def get(self):
            return self.a


    class TestInstanceCountExceptionerB(metaclass=InstanceCountExceptioner):
        __max_instance_count__ = 3

        def __init__(self):
            self.b = 2

        def get(self):
            return self.b
        
    a_one = TestInstanceCountExceptionerA()
    print(a_one.a)
    
    a_two = TestInstanceCountExceptionerA()
    print(a_one.a)
    
    try:
        a_three = TestInstanceCountExceptionerA()
    except InstanceCountException as e:
        print(e)
        
    
test_fail_create_a()

1
1
Reached max number of instances


In [14]:
def test_fail_create_b():
    class TestInstanceCountExceptionerA(metaclass=InstanceCountExceptioner):
        __max_instance_count__ = 2

        def __init__(self):
            self.a = 1

        def get(self):
            return self.a


    class TestInstanceCountExceptionerB(metaclass=InstanceCountExceptioner):
        __max_instance_count__ = 3

        def __init__(self):
            self.b = 2

        def get(self):
            return self.b
        
    b_one = TestInstanceCountExceptionerB()
    print(b_one.b)
    
    b_two = TestInstanceCountExceptionerB()
    print(b_one.b)
    
    b_three = TestInstanceCountExceptionerB()
    print(b_one.b)
    
    try:
        b_four = TestInstanceCountExceptionerB()
    except InstanceCountException as e:
        print(e)
        
    
test_fail_create_b()

2
2
2
Reached max number of instances


## 3. JSONClassCreator (0.6 баллов)
Напишите метакласс, который будет по json представлению строить новый класс и обратно. Класс должен уметь следующее:
* Поддерживать сохранение и получение магических функций класса.
* Поддерживать сохранение и получение обычных функций.
* Поддерживать сохранение полей со стандартными типами.
* Уберите из сохранения следующие поля и методы: ['\_\_dict\_\_', '\_\_weakref\_\_', '\_\_module\_\_', '\_\_init\_\_']
* У создаваемого класса должна быть функция to_json_str

Формат json строки должен быть следующий:

<code>
{
    "name": название класса,
    "bases": базовые классы,
    "methods": методы класса,
    "attrs": поля класса
}
</code>

Рекомендации:
* Для получения кода функций используйте модуль <a href="http://python-lab.ru/documentation/27/stdlib/inspect.html">inspect</a>.
* Для того, чтобы запустить код функций, можно использовать exec.
* Можно не исправлять ошибку типа OSError: could not get source code - возникает для функций, полученных с помощью exec.

#### Пример использования

In [ ]:
import json


class ParentTest1(object):
    pass

class ParentTest2(object):
    pass

class Test(ParentTest1, ParentTest2):
    """Тестовый класс"""

    val = [1, 2, 3]

    def f(self, x):
        print(x)
    
    def __repr__(self):
        return "Test(val={})".format(self.val)

    def __str__(self):
        return "Test(val={})".format(self.val)

    pass

In [ ]:
print(*json.loads(JSONClassCreator.to_json_str(Test)).items(), sep="\n")

In [ ]:
tmp = JSONClassCreator(JSONClassCreator.to_json_str(Test))

In [ ]:
tmp_obj = tmp()
tmp_obj, tmp_obj.f("hi"), tmp.val, tmp.__doc__

In [ ]:
tmp.__dict__

#### Решение

In [ ]:
import inspect


class JSONClassCreator(type):
    def __new__(mcls, json_str):
        <your code here>

    def to_json_str(cls):
        exclude = ['__dict__', '__weakref__', '__module__', '__init__']
        <your code here>

        return json.dumps({
            "name": <your code here>,
            "bases": <your code here>,
            "methods": <your code here>,
            "attrs": <your code here>
        })

    pass


#### Проверьте свое решение на примере